In [ ]:
import matplotlib.pyplot as plt  
import pandas as pd  
import numpy as np 
import seaborn as sns
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

In [ ]:
level = 2
setup = 'oversigma_10tissue'
#label = 'disease_type'
label='primary_site'
#label = 'uniq'
L = 3
#labels = ['primary_site', 'disease_type']
labels = ['primary_site', 'secondary_site']
#directory = "results/hSBM/%s"%setup
directory="gtex/hsbm/%s"%setup
df = pd.read_csv("%s/mainTable.csv"%(directory), index_col=[0], header=[0])
df.head()

In [ ]:
data=df.T.values[:1183]

In [ ]:
data.shape

In [ ]:
dend = shc.dendrogram(shc.linkage(data, method='ward'))

In [ ]:
cluster = AgglomerativeClustering(n_clusters=9, affinity='euclidean', linkage='ward')  
out = cluster.fit_predict(data)

In [ ]:
from hsbmpy import define_labels, get_cluster_given_l

In [ ]:
true_labels, predicted_labels = define_labels(get_cluster_given_l(2,directory),pd.read_csv("%s/files.dat"%directory, index_col=[0]), label='primary_site')

In [ ]:
np.shape(out)

In [ ]:
np.shape(predicted_labels)

In [ ]:
np.shape(true_labels)

In [ ]:
from sklearn.metrics import homogeneity_completeness_v_measure

In [ ]:
homogeneity_completeness_v_measure(true_labels,predicted_labels)

In [ ]:
df_files = pd.read_csv("%s/files.dat"%directory, index_col=[0])

In [ ]:
from hsbmpy import get_file

In [ ]:
scores = {}
scores['hierarchical']={
    'h':[],
    'c':[],
    'V':[]
}

In [ ]:
for x in [1183, 268, 34, 10]:
    cluster = AgglomerativeClustering(n_clusters=x, affinity='euclidean', linkage='ward')  
    out = cluster.fit_predict(data)
    true_out = []
    for sample in df.columns.values[0:1183]:
        true_out.append(get_file(sample, df_files)['primary_site'])
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['hierarchical']['h'].append(score[0])
    scores['hierarchical']['c'].append(score[1])
    scores['hierarchical']['V'].append(score[2])

In [ ]:
scores

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda = LatentDirichletAllocation(n_components=100,random_state=42)
topics = lda.fit_transform(data)

In [ ]:
cm = sns.clustermap(topics)

In [ ]:
cluster.fit_predict(topics)

In [ ]:
scores['lda']={
    'h':[],
    'c':[],
    'V':[]
}
for x in [1183, 268, 34, 10]:
    cluster = AgglomerativeClustering(n_clusters=x, affinity='euclidean', linkage='ward')  
    out = cluster.fit_predict(topics)
    true_out = []
    for sample in df.columns.values[0:1183]:
        true_out.append(get_file(sample, df_files)['primary_site'])
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['lda']['h'].append(score[0])
    scores['lda']['c'].append(score[1])
    scores['lda']['V'].append(score[2])

In [ ]:
scores['lda']

In [8]:
import tensorflow as tf
import pandas as pd

In [11]:
df = pd.read_csv("mainTable.csv", index_col=[0], header=[0])

In [85]:
tfdf = tf.convert_to_tensor(df.transpose().values, dtype=tf.float64)
tfdf.shape

TensorShape([Dimension(407), Dimension(223)])

In [84]:
def euclidean_self(input_tensor):
    tensor_shape = input_tensor.get_shape().as_list()
    tensor_iter = input_tensor
    euclidean_list = []

    ses = tf.Session()

    for i in range(tensor_shape[0]):
        if i%100==0:
            print(i)
        split_head, split_tail = tf.split(tensor_iter, [1, tensor_shape[0]-1])
        tensor_iter = tf.concat([split_tail, split_head], 0)
        euclidean_dist = (tf.sqrt(tf.reduce_sum(tf.square(input_tensor-tensor_iter), 1)))
        ses.run(euclidean_dist)
        euclidean_row = euclidean_dist.eval(session=ses)
        euclidean_list.append(euclidean_row)

    ses.close()

    euclidean_out = np.asarray(euclidean_list).transpose([1, 0])

    for i in range(tensor_shape[0]):
        euclidean_out[i] = np.append(euclidean_out[i][tensor_shape[0]-1-i:], 
                                     euclidean_out[i][:tensor_shape[0]-1-i])

    '''
    the return type is numpy.array, if need tensor for return, use the code beneath
    '''
    # euclidean_out = tf.convert_to_tensor(euclidean_out)

    return euclidean_out

In [86]:
euclidean_self(tfdf)

0
100
200
300
400


NameError: name 'np' is not defined

In [79]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(alternate))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [46]:
df.head()

,d2587070-cb7d-440d-ae49-52f5077248e6.htseq.counts.gz,97675f21-ce94-4e7c-bc46-179cc2903bf3.htseq.counts.gz,646d58e7-b7e2-4b2b-bb37-3ca20b601213.htseq.counts.gz,0f2b58b4-7d91-48f3-8901-b624e3d6aa51.htseq.counts.gz,3eb03db7-efc1-4f20-bbc9-61551905eb9b.htseq.counts.gz,73ce143e-44f1-49c6-963f-45246d8551de.htseq.counts.gz,09f2bb20-909e-4929-8a79-c64c7fb2af31.htseq.counts.gz,6ae2dd6c-2a39-411f-a1fc-11e0e6e82165.htseq.counts.gz,5f5c42f3-3631-43a3-9774-164eca819389.htseq.counts.gz,b520edf2-42ce-4b64-8575-19e34b30f042.htseq.counts.gz,...,37e498a7-b147-4c38-a3f5-ed283d57f97a.htseq.counts.gz,c264fe3b-482b-44ec-83a4-73df565663ff.htseq.counts.gz,9e97f017-6855-4d68-bd2a-c4da3ccb1d57.htseq.counts.gz,15864159-be88-41c8-bdef-c2c5927cb1a1.htseq.counts.gz,e2bdca76-7bc5-4143-a6f6-3c6642425c65.htseq.counts.gz,b1868d9d-ef06-4353-883b-230dae23f8b6.htseq.counts.gz,2b5f6764-5a5b-47dc-b0f8-7a548bab2e26.htseq.counts.gz,7324b2d6-dd30-4f99-a101-7c3d497bbcb8.htseq.counts.gz,95e726db-5ccc-4836-a2ae-7feaddaf9f1b.htseq.counts.gz,00cc9b4d-a847-464e-979a-7751e1a87ae3.htseq.counts.gz
ENSG00000276289,0,0,0,0,4,0,1,0,0,0,...,0,0,0,25,2,0,1,1,5,1
ENSG00000137948,1,0,0,0,1,0,0,1,2,3,...,2,0,0,0,0,0,1,2,0,0
ENSG00000057149,0,0,0,115,6,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,1
ENSG00000130287,7,2,0,6,1,2,4,1,2,24,...,4,2,1,0,0,1,0,1,7,0
ENSG00000113492,0,1,0,5,0,0,1,1,0,1,...,0,2,0,0,0,3,1,1,3,0


In [55]:
tfdf[0].shape

TensorShape([Dimension(223)])